# \[FRAUD\] 데이터 정리 시도(8.18-망함 df50을 tr/test로 분리했다가 다시

합쳐봄..)

김보람  
2023-08-18

# imports

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import torch

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics 

# embedding 
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder

In [77]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df["cc_num"].values.tolist()+\
                                                      df["merchant"].values.tolist()))}
    
    df["from"]=df["cc_num"].apply(lambda x:mapping[x])  #엣지의 출발점
    df["to"]=df["merchant"].apply(lambda x:mapping[x])  #엣지의 도착점
    
    df = df[['from', 'to', "amt", "is_fraud"]].groupby(['from','to']).agg({"is_fraud":"sum","amt":"sum"}).reset_index()
    df["is_fraud"]=df["is_fraud"].apply(lambda x:1 if x>0 else 0)
    
    G=nx.from_edgelist(df[["from","to"]].values, create_using=graph_type)
    
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["is_fraud"] for idx, x in df[["from","to","is_fraud"]].iterrows()}, "label")  #엣지 속성 설정,각 속성의 사기 여부부     
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["amt"] for idx,x in df[["from","to","amt"]].iterrows()}, "weight") # 엣지 속성 설정, 각 엣지의 거래 금액

    return G


def build_graph_tripartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df.index.values.tolist() + 
                                                       df["cc_num"].values.tolist() +
                                                       df["merchant"].values.tolist()))}
    df["in_node"]= df["cc_num"].apply(lambda x: mapping[x])
    df["out_node"]=df["merchant"].apply(lambda x:mapping[x])
    
        
    G=nx.from_edgelist([(x["in_node"], mapping[idx]) for idx, x in df.iterrows()] +\
                        [(x["out_node"], mapping[idx]) for idx, x in df.iterrows()], create_using=graph_type)
    
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")     
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")   
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")  
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")

    return G
    
    
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def embedding(Graph):
    # Graph -> X (feature)
    _edgs = list(Graph.edges)
    subGraph = Graph.edge_subgraph([_edgs[x] for x in range(len(Graph.edges))]).copy()
    subGraph.add_nodes_from(list(set(Graph.nodes) - set(subGraph.nodes)))    
    embedded = AverageEmbedder(Node2Vec(subGraph, weight_key='weight').fit(window=10).wv)
    X = [embedded[str(_edgs[x][0]), str(_edgs[x][1])] for x in range(len(Graph.edges))]
    # Graph -> y (label)
    y = np.array(list(nx.get_edge_attributes(Graph, "label").values()))
    return X,y 

def anal(df):
    Graph = build_graph_bipartite(df)
    X,XX,y,yy = embedding(Graph)
    lrnr = RandomForestClassifier(n_estimators=100, random_state=42) 
    lrnr.fit(X,y)
    yyhat = lrnr.predict(XX)
    df = pd.DataFrame({
        'acc':[sklearn.metrics.accuracy_score(yy,yyhat)], 
        'pre':[sklearn.metrics.precision_score(yy,yyhat)], 
        'rec':[sklearn.metrics.recall_score(yy,yyhat)],
        'f1':[sklearn.metrics.f1_score(yy,yyhat)]}
    )    
    return df

def our_sampling1(df):
    cus_list = set(df.query('is_fraud==1').cc_num.tolist())
    return df.query("cc_num in @ cus_list")

In [78]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [79]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## 시도

In [80]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [81]:
df50 = down_sample_textbook(df02)
df50.shape

In [82]:
12012*12012

### 고려할 것(230810)

-   df50 의 shape이 12000개 이므로 9000개의 T, 3000개의 F를 train mask로
    만들자.

-   고객정보가 동일하면 edge를 1로, 아니면 0으로 놓고 1에대한 weight를
    만들자.

-   g(V,E,W)에서의 weight

In [83]:
df50 = df50.reset_index()

In [84]:
N = len(df50)

------------------------------------------------------------------------

### tr/test

In [11]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [12]:
df50_tr.is_fraud.mean().round(5), df50_test.is_fraud.mean().round(5)

In [13]:
df50_tr.shape, df50_test.shape

In [14]:
train_mask = np.concatenate((np.full(9009, True), np.full(3003, False)))
test_mask = np.concatenate((np.full(9009, False), np.full(3003, True)))
print("Train Mask:", train_mask)
print("Test Mask:", test_mask)

Train Mask: [ True  True  True ... False False False]
Test Mask: [False False False ...  True  True  True]

In [15]:
train_mask.shape, test_mask.shape

In [16]:
train_mask.sum(), test_mask.sum()

In [25]:
df50_com = pd.concat([df50_tr, df50_test])

In [27]:
df50_com = df50_com.reset_index()

In [28]:
df50_com

------------------------------------------------------------------------

### 이건 weight?

In [ ]:
# edge_index_list = []
# for i in range(N):
#     for j in range(N):
#         time_difference = (df50['trans_date_trans_time'][i] - df50['trans_date_trans_time'][j]).total_seconds()
#         edge_index_list.append([i, j, time_difference])

In [ ]:
# edge_index_list[:5]

In [ ]:
# np.save('edge_index_list_50.npy', edge_index_list)

# loaded_data = np.load('edge_index_list_50.npy')

In [ ]:
# edge_index = np.array(edge_index_list)
# edge_index[:,2] = np.abs(edge_index[:,2])
# theta = edge_index[:,2].mean()
# theta

In [ ]:
# edge_index[:,2] = (np.exp(-edge_index[:,2]/theta)!=1) * np.exp(-edge_index[:,2]/theta)
# edge_index

In [ ]:
# edge_index[:,2]

Q. 그런데 밑에서 random으로 train하고 test로 나누게 되면.. wieght랑
edge를 어떻게 적용시키지?

### edge: 같은 cc_num이면 edge=1, 다르면 edge=0

In [36]:
edge_index_list2_com = []
for i in range(N):
    for j in range(N):
        if df50_com['cc_num'][i] != df50_com['cc_num'][j]:  
            edge = 0
        else:
            edge = 1
        edge_index_list2_com.append([i, j, edge])

In [40]:
np.save('edge_index_list2_50_com.npy', edge_index_list2_com)

loaded_data = np.load('edge_index_list2_50_com.npy')

In [38]:
edge_index_list2_com[:5]

In [39]:
edge_one_com = [(i, j) for i, j, edge in edge_index_list2_com if edge == 1]
edge_one_com[:5]

In [41]:
len(edge_one_com)

In [42]:
edge_one_index_com = torch.tensor(edge_one_com, dtype=torch.long).t()

In [43]:
edge_one_index_com.shape

------------------------------------------------------------------------

### data설정(x, edge_index, y)

In [44]:
x = df50_com['amt']

In [45]:
a = torch.tensor(x, dtype=torch.float)

In [46]:
a = a.reshape(-1,1)
a

In [47]:
y = df50_com['is_fraud']

In [48]:
b = torch.tensor(y,dtype=torch.int64)

In [49]:
b

In [50]:
import torch_geometric

In [51]:
data = torch_geometric.data.Data(x=a, edge_index = edge_one_index_com, y=b, train_mask = train_mask, test_mask = test_mask)

In [52]:
data

------------------------------------------------------------------------

------------------------------------------------------------------------

### gnn

In [53]:


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [54]:
model = GCN()

In [55]:
model

In [56]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()

------------------------------------------------------------------------

In [57]:
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [58]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.5012

> 왜 자꾸 accuracy가 안좋게 나오노

In [65]:
data.y[data.test_mask].sum()

In [70]:
out[data.train_mask]

In [71]:
data.y[data.test_mask]

> 음……………. edge_list를 다시 해보자.

> !!!! 첫번째 시도에서는 edge_list를 무작정 1인걸 고른것이 아니였는데 왜
> 여기서는 이러고 있엇니……………..